In [ ]:
import tqdm
# Autoreload possibly interferes with IntelliJ debugging
%reload_ext autoreload
%autoreload 2
import logging
class FlushHandler(logging.StreamHandler):
    def emit(self, record):
        super().emit(record)
        self.flush()

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[FlushHandler()])
log = lambda msg: logging.info(msg)


# Full pipeline (multiple files)

In [ ]:
import pandas as pd
import os

input_dir = "C:\\dev\\play\\brainwave-data-day"
stats_df = pd.read_csv(input_dir + os.path.sep + "day_stats.csv")
# stats_df = pd.read_csv("C:\\dev\\play\\brainwave-data\\stats.csv")

In [ ]:
# import models.eeg_states.eeg_states
# reload(models.eeg_states.eeg_states)
# from models.eeg_states.eeg_states import load_and_prepare_day_data_energy_eeg_state_events
#
# events = load_and_prepare_day_data_energy_eeg_state_events()

## Convert Brainflow files to FIF

In [ ]:
def get_brainflow_compressed_filename(full_input_filename: str) -> str:
    full_output_dirname = webserver.output_dirname(full_input_filename)
    compressed_full_output_filename = str(os.path.join(full_output_dirname, os.path.basename(full_input_filename))) + '.bz2'
    return compressed_full_output_filename

In [ ]:
from datetime import datetime

import webserver
import convert
# import zstandard as zstd
import os
import bz2
import time
import shutil
from tqdm.notebook import trange, tqdm

errors = []
processed = []

# Could get these working later
skip_list = []

force = False

def compress_bz2(input_file, output_file):
    start_time = time.time()
    with open(input_file, 'rb') as f_in:
        with bz2.open(output_file, 'wb', compresslevel=9) as f_out:
            shutil.copyfileobj(f_in, f_out)
    end_time = time.time()
    return end_time - start_time, os.path.getsize(output_file)

for root, dirs, files in os.walk(input_dir):
    # Exclude the last file, which we assume to be the most recent, and possibly still being written
    files = [file for file in files if file.endswith(".brainflow.csv")][:-1]
    for idx, file_name in tqdm(enumerate(files), desc="Processing directories", total=(len(files))):  
        full_input_filename = os.path.join(root, file_name)
        try:
            full_output_dirname = webserver.output_dirname(full_input_filename)
            full_output_filename = str(os.path.join(full_output_dirname, 'raw.fif'))
            
            compressed_full_output_filename = get_brainflow_compressed_filename(full_input_filename)
            
            if not os.path.exists(compressed_full_output_filename) or force:
                log(f"Compressing file {full_input_filename} to " + compressed_full_output_filename)
                processed.append("Compressing " + full_input_filename)
                try:
                    os.mkdir(os.path.dirname(compressed_full_output_filename))
                except:
                    pass
                compress_bz2(full_input_filename, compressed_full_output_filename) 
                
            if os.path.exists(full_output_filename) and not force:
                log(f"Skipping file {full_input_filename} as {full_output_filename} and {compressed_full_output_filename} already exist")
                continue
            should_skip = False
            for s in skip_list:
                if s in full_input_filename:
                    log(f"Skipping file {full_input_filename}")
                    should_skip = True
            if not should_skip:
                log(f"Processing file {full_input_filename}")
                processed.append("Processing " + full_input_filename)
                channels = ['Fpz-M1']
                date_time_str = os.path.basename(full_input_filename).removesuffix(".brainflow.csv")
                date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
    
                if channels is not None:
                    log(f"Processing file {full_input_filename} with channels {channels}")
                    convert.convert_and_save_brainflow_file_with_gap_filling(log, full_input_filename, full_output_filename, channels)

        except Exception as e:
            msg = "Error processing file: " + full_input_filename
            log(msg)
            log(e)
            errors.append(msg)


In [ ]:

errors

In [ ]:
processed

## Run pipeline on FIF files

In [ ]:
import contextlib
import io
import run_day_pipeline
from importlib import reload
reload(run_day_pipeline)
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import trange, tqdm

errors = []
dataframes = []

# Could get these working later
skip_list = []

def process_file(root, dir_name):
    input_file = os.path.join(root, dir_name, "raw.fif")
    if dir_name in skip_list:
        log(f"Skipping {dir_name}: " + input_file)
        return None
    try:
        log(f"Processing file: " + input_file)
        if os.path.exists(input_file):
            yasa_df = run_day_pipeline.cached_pipeline(log, input_file, stats_df, events)
            #log(f"Returning {yasa_df.head()}")
            return yasa_df
    except Exception as e:
        msg = f"Error processing file: " + input_file + " - " + str(e)
        log(msg)
        errors.append(msg)
        log(e)
    return None

for root, dirs, files in os.walk(input_dir):
    for dir_name in tqdm(dirs, desc="Processing directories", total=len(dirs)):
        output_buffer = io.StringIO()
        with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
            df = process_file(root, dir_name)
            if df is not None:
                dataframes.append(df)

all = pd.concat(dataframes)
log(f"Finished processing, have {len(all)} files in total")

In [ ]:
dataframes

In [ ]:
errors

# Recalculate scalings
N.b. can be run frequently but will only be picked up by new runs.  Maybe worth occasionally regenerating all old files.
And yes, for new features have to rerun the pipeline on everything, then generate the stats here, then rerun the pipeline again on everything to have them use those.

In [ ]:
from scaling import only_eeg

only_eeg_cols = list(only_eeg(all).columns)
assert any(col.startswith("Main") for col in only_eeg_cols), "No column starting with 'Main' found in only_eeg_cols"

In [ ]:
import scaling

stats = scaling.stats(all)
stats.to_csv(input_dir + "/day_stats.csv")
assert any(stats['Column'].str.startswith("Main")), "No row starting with 'Main' found in column_name"
stats

# Upload to GCS

In [ ]:
from upload import upload_dir_to_gcs_skipping_existing
import os

errors = []
dataframes = []

for root, dirs, files in os.walk(input_dir):
    for dir_name in tqdm(reversed(dirs), desc="Uploading directories", total=len(dirs)):
        full_dir_name = os.path.join(root, dir_name)
        try:
            upload_dir_to_gcs_skipping_existing(log, 'examined-life-input-eeg-day', full_dir_name, dir_name)
        except Exception as e:
            log("Error processing file: " + input_dir)
            log(e)

for error in errors:
    log(error)

log("All uploaded")

# Check if can delete Brainwave files that are safely backed up

In [ ]:

import os

errors = []
can_delete = []
cannot_delete = []

for root, dirs, files in os.walk(input_dir):
    for idx, file_name in enumerate(files):
        full_input_filename = os.path.join(root, file_name)
        if full_input_filename.endswith(".brainflow.csv"):
            compressed_full_output_filename = get_brainflow_compressed_filename(full_input_filename)
                
            if os.path.exists(compressed_full_output_filename):
                can_delete.append({
                    'backed_up': compressed_full_output_filename,
                    'full_filename': full_input_filename
                })
            else:
                cannot_delete.append(full_input_filename)
    

In [ ]:
can_delete_df = pd.DataFrame(can_delete)
can_delete_df

In [ ]:
cannot_delete

In [ ]:
can_delete_df['full_filename']